In [11]:
import sys
sys.path.insert(0, '../src/')

import os
import abc
import pickle
import warnings
warnings.filterwarnings('ignore')

import time

#import tensorflow as tf
import torch
device = 'cpu'
dtype = torch.float32

import scipy.sparse as sp
import numpy as np
from matplotlib import pyplot as plt
from scipy.sparse import save_npz, load_npz, csr_matrix
from sklearn.metrics import roc_auc_score, average_precision_score
import time
import pandas as pd
%matplotlib inline

# from netgan.netgan import *
# from netgan import utils

from net.utils import *
from net import utils_netgan as utils
import net.net as net

In [77]:
class Evaluation(object):
    def __init__(self, experiment_root, statistic_fns):
        self.experiment_root = experiment_root
        self.statistic_fns = statistic_fns
        
    def _load_timing(self, experiment):
        return self._load(experiment, 'timing')
    
    def _load_overlap(self, experiment):
        return self._load(experiment, 'overlap')
    
    def _load(self, experiment, name):
        filename = os.path.join(self.experiment_root,
                                f'Experiment_{experiment}/sampled_graphs/{name}.pickle')
        with open(filename, 'rb') as handle:
            timing = pickle.load(handle)
        return timing
    
    def compute_statistics(self):
        # parse experiment root folder
        num_experiments = len(os.listdir(self.experiment_root))
        # load overlaps and timings
        overlaps = [self._load_overlap(i) for i in range(num_experiments)]
        timings = [self._load_timing(i) for i in range(num_experiments)]
        
        steps = max(timings[0].keys())
        step_len = int(np.log10(steps))+1
        invoke_every = steps // len(timings[0].keys())
        
        statistics = {name: np.zeros([num_experiments, steps//invoke_every]) for name in self.statistic_fns.keys()}
        statistics['Edge Overlap (%)'] = np.zeros([num_experiments, steps//invoke_every])
        statistics['Time (s)'] = np.zeros([num_experiments, steps//invoke_every])
        for step in range(invoke_every, steps, invoke_every):
            for experiment in range(num_experiments):
                # load sparse graph
                graph_name = f'graph_{step:0{step_len}d}.npz'
                graph_path = os.path.join(self.experiment_root,
                                          f'Experiment_{experiment}/sampled_graphs/{graph_name}')
                graph = load_npz(graph_path)
                # compute statistics
                statistics['Edge Overlap (%)'][experiment, step//invoke_every] = overlaps[experiment][step]
                statistics['Time (s)'][experiment, step//invoke_every] = timings[experiment][step]
                for name, statistic_fn in self.statistic_fns.items():
                    statistics[name][experiment, step] = statistic_fn(graph)
        self.statistics = statistics
        self.steps = steps
        self.invoke_every = invoke_every
                    
                    

    def export_statistics(self):
        pass
    def plot_statistics(self):
        pass

In [78]:
eval_exp = Evaluation(experiment_root='../logs/experiments_CORA-ML/',
                      statistic_fns={})

In [79]:
eval_exp.compute_statistics()

In [80]:
eval_exp.statistics

{'Edge Overlap (%)': array([[0.        , 0.00383311, 0.0984815 , 0.33554474, 0.43785935,
         0.54577621, 0.61683621, 0.66755123, 0.71118974, 0.74214949,
         0.76382132, 0.78888397, 0.80377414, 0.81836945, 0.82603568,
         0.83782987, 0.85404688, 0.85994398, 0.86878962, 0.87453929],
        [0.        , 0.00530739, 0.08712959, 0.34542238, 0.46144774,
         0.55064131, 0.62686127, 0.67271119, 0.72077252, 0.75114256,
         0.77001327, 0.79640277, 0.81158779, 0.81409406, 0.8369453 ,
         0.84505381, 0.85286746, 0.8659885 , 0.87394958, 0.87969925]]),
 'Time (s)': array([[ 0.        ,  2.88219547,  5.77191591,  8.67122793, 11.55827117,
         14.36798763, 17.18894267, 20.03583026, 22.86395001, 25.69589043,
         28.49020672, 31.28385186, 34.09897828, 36.90437508, 39.70815325,
         42.52123237, 45.35031343, 48.18264437, 51.09562325, 53.98334503],
        [ 0.        ,  2.8838408 ,  5.78501344,  8.69969201, 11.55391216,
         14.39598751, 17.21862197, 20.052

In [38]:
print('%')

%


In [25]:
help(load_npz)

Help on function load_npz in module scipy.sparse._matrix_io:

load_npz(file)
    Load a sparse matrix from a file using ``.npz`` format.
    
    Parameters
    ----------
    file : str or file-like object
        Either the file name (string) or an open file (file-like object)
        where the data will be loaded.
    
    Returns
    -------
    result : csc_matrix, csr_matrix, bsr_matrix, dia_matrix or coo_matrix
        A sparse matrix containing the loaded data.
    
    Raises
    ------
    IOError
        If the input file does not exist or cannot be read.
    
    See Also
    --------
    scipy.sparse.save_npz: Save a sparse matrix to a file using ``.npz`` format.
    numpy.load: Load several arrays from a ``.npz`` archive.
    
    Examples
    --------
    Store sparse matrix to disk, and load it again:
    
    >>> import scipy.sparse
    >>> sparse_matrix = scipy.sparse.csc_matrix(np.array([[0, 0, 3], [4, 0, 0]]))
    >>> sparse_matrix
    <2x3 sparse matrix of type '<c

In [19]:
dct = {1:'a', 2:'b', 5:'e'}

In [21]:
max(dct.keys())

5

In [32]:
os.listdir('../logs/experiments_CORA-ML/Experiment_0/sampled_graphs/')

['graph_090.npz',
 'graph_080.npz',
 'graph_100.npz',
 'overlap.pickle',
 'graph_060.npz',
 'graph_015.npz',
 'graph_075.npz',
 'graph_005.npz',
 'graph_020.npz',
 'graph_070.npz',
 'graph_065.npz',
 'graph_010.npz',
 'graph_040.npz',
 'graph_095.npz',
 'graph_050.npz',
 'graph_035.npz',
 'timing.pickle',
 'graph_025.npz',
 'graph_085.npz',
 'graph_055.npz',
 'graph_045.npz',
 'graph_030.npz']

In [2]:
def timing(f):
    def g(*args, **kwargs):
        start = time.time()
        y = f(*args, **kwargs)
        g.last_time = time.time() - start
        return y
    return g

In [5]:
@timing
def loop(iters):
    x = 0
    for i in range(iters):
        x += i
    return x

In [9]:
loop(100)
loop.last_time

1.3828277587890625e-05

In [12]:
def f(*args, **kwargs):
    y = 0
    for x in args:
        y += x
    return y

In [16]:
lst = [1,2,3,4,5,6]
f(*lst)

21

In [ ]:
f(lst[0], lst[1])